# Example notebook on how to calculate RVs from CCFs using different line masks

In [ ]:
import sys

sys.path.append('/Users/tervin/NEID_Solar_analysis')

import os
import glob
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import constants
from sunpy.net import attrs as a

from astropy.io import fits

import NEIDcode

import tamar.tools.ccf_funcs as ccffuncs
from tamar.tools.settings import CsvDir, Config

File names for saving CCF information (ccf_csv) and csv file
with SDO calculations (sdo_csv).

Names of mask and fsr mask to use.
- specify if the fsr mask needs to be inverted for use

In [ ]:
# csv file to save ccf information
ccf_csv = os.path.join(CsvDir.CCFS, 'ccfs.csv')

# csv with sdo calculation
sdo_csv = os.path.join(CsvDir.NEID_CALC, 'rvs_from_fits.csv')

# mask to use
mask_name = 'G2_espresso.txt'

# fsr mask -- None if no fsr mask
fsr_mask = CsvDir.FSR_MASK
invert = True  # True if fsr mask is inverted

# get list of dates!
dates = os.listdir(CsvDir.NEID_SOLAR)  # this is the folder storing the NEID directory structure
dates = sorted(dates)

Configuration parameters.
- No updates needed.

In [ ]:
# configuration parameters
config = Config.config

# parameters for SDO/HMI image generation
time_range = datetime.timedelta(seconds=22)
physobs_list = [a.Physobs.los_velocity, a.Physobs.los_magnetic_field, a.Physobs.intensity]

# lightspeed
LIGHTSPEED = constants.c / 1000  # Speed of light in km/s
minzb = -30 / LIGHTSPEED
maxzb = +30 / LIGHTSPEED

# Set up velocity loop
velocity_loop = np.arange(config['velocity_min'], config['velocity_max'], config['velocity_step']) + config['qrv']
velsize = len(velocity_loop)

Generation of mask object using mask file and config parameters.

In [ ]:
# generate mask object based on config parameters
mask = NEIDcode.Mask(mask_name, config['mask_half_width'], config['mask_environment'])
fsr_mask = fits.getdata(fsr_mask)
if invert:
    fsr_mask = 1 - fsr_mask

In [ ]:
# read in SDO calculations csv
df = pd.read_csv(sdo_csv)
sdo_dates = df.date_obs.values

Loop through dates to calculate and save CCF RV information to specified
csv file.

In [ ]:
# make lists for saving
rv_sun, rv_model, error, ccfs, shift_ccfs, gaussian, rv_gauss, Bobs, v_conv = [], [], [], [], [], [], [], [], []
for date in dates:
    # get neid files
    file = os.path.join(CsvDir.NEID_SOLAR, date, 'level2', date)
    files = [i for i in glob.glob(os.path.join(file, '*.fits'))]
    nfiles = len(files)
    print('Calculating Weighted CCF for', date, 'using', nfiles, 'files.')

    # total number of extracted orders
    orders = 122  # hard coded for NEID

    # setup CCF calculation
    ccfs_fortran = np.zeros([nfiles, orders, velsize])
    ccfs_pipeline = np.zeros([nfiles, orders, velsize])
    rv_order = np.zeros([nfiles, orders])
    ccf_weighted = np.zeros([nfiles, velsize])
    file_ccfs = np.zeros([nfiles, velsize])

    # calculate CCFs and RVs
    rv_in_file = []
    dvrms = []
    for n, f in enumerate(files):
        fd = fits.open(f)
        flux = fd['SCIFLUX'].data
        wave = fd['SCIWAVE'].data
        head = fd[0].header
        ccfhead = fd['CCFS'].header
        ccfwt = np.zeros(122)
        main_header = fits.getheader(f)
        date_jd = fits.getheader(f)['OBSJD']

        if main_header['DRIFTFUN'] == 'simultaneous' and main_header['WAVECAL'] == 'LFCplusThAr':
            # get RV and error from SDO calculations
            date_obs = date[0:4] + "-" + date[4:6].zfill(2) + '-' + date[6:8].zfill(2) + 'T12:00:00'
            sdo_inds = np.isin(sdo_dates, date_obs)
            rv = df.rv_model_weather_coeff.values[sdo_inds] / 1e3
            B = df.Bobs.values[sdo_inds]
            vconv = df.v_conv.values[sdo_inds]
            # check to even see if we should continue
            if len(rv) == 0:
                pass
            else:
                # check that uncertainty is not terrible
                if fits.getheader(f, 'CCFS')['DVRMS'] <= .0004:
                    # switch between order index and echelle order, then loop
                    rv_in_file.append(fits.getheader(f, 'CCFS')['CCFRVMOD'])
                    dvrms.append(fits.getheader(f, 'CCFS')['DVRMS'])
                    for trueorder in tqdm(range(config['ordmin'], config['ordmax'] + 1, 1)):
                        zb = head['SSBZ%03i' % trueorder]
                        berv = head['SSBRV%03i' % trueorder]
                        raworder = config['bluest_order'] - trueorder
                        ccfwt[raworder] = ccfhead['CCFWT%03i' % trueorder]

                        # You have to remove NaNs ahead of passing spectrum to Fortran code
                        nanfree = NEIDcode.remove_nans(flux[raworder, :], method='linear')
                        spectrum = nanfree[0]

                        if fsr_mask is None:
                            # Number of blaze edge pixels to clip on either side of order
                            pix_start = config['clip_edge_pixels']
                            pix_end = np.shape(flux)[1] - config['clip_edge_pixels']
                        else:
                            if np.nansum(np.logical_not(fsr_mask[raworder, :])) == 0:
                                print('fsr mask order summed to zero...sad')
                                continue
                            else:
                                pix_start = np.min(np.argwhere(np.logical_not(fsr_mask[raworder, :])))
                                pix_end = np.max(np.argwhere(np.logical_not(fsr_mask[raworder, :])))

                        dummy_line_start = mask.start * ((1.0 + (velocity_loop[0] / LIGHTSPEED)) / (1.0 + maxzb))
                        dummy_line_end = mask.end * ((1.0 + (velocity_loop[-1] / LIGHTSPEED)) / (1.0 + minzb))
                        try:
                            line_index = np.where((dummy_line_start > np.min(wave[raworder, pix_start:pix_end])) &
                                                  (dummy_line_end < np.max(wave[raworder, pix_start:pix_end])))[0]
                        except TypeError:
                            print('Line index is None...devastating')
                            line_index = []

                        sn = np.ones(len(flux[raworder, pix_start:pix_end]))
                        if not len(line_index) == 0:
                            for k in range(len(velocity_loop)):
                                ccfs_fortran[n, raworder, k] = NEIDcode.CCF_3d.ccf(mask.start[line_index],
                                                                                   mask.end[line_index],
                                                                                   wave[raworder, pix_start:pix_end],
                                                                                   spectrum[pix_start:pix_end],
                                                                                   mask.weight[line_index],
                                                                                   sn, velocity_loop[k], berv, 0.)
                        else:
                            ccfs_fortran[n, raworder, :] = np.zeros(len(velocity_loop))

            # check again that we have sdo data
            if len(rv) == 0:
                pass
            else:
                # calculate full CCF
                ccfsum = np.nansum(ccfs_fortran[n, :, :], axis=0)

                # apply weighting scheme to each order to get ccf from each file
                ccfmod = np.nansum((ccfs_fortran[n, :, :].T * ccfwt).T, axis=0)
                rescale = np.nansum(ccfsum) / np.nansum(ccfmod)
                ccfmod *= rescale
                file_ccfs[n, :] = ccfmod

    # sum up all of the weighted CCFs to make the 'final' CCF
    ccf = np.nansum(file_ccfs, axis=0)

    # normalize
    ccf /= np.nanmax(ccf)

    # get gaussian fit
    gaussian_fit, g_x, g_y, final_rv = ccffuncs.fit_gaussian_to_ccf(velocity_loop, ccf, config['qrv'],
                                                                    config['velocity_half_range_to_fit'])

    # add these to lists
    rv_model.append(rv)
    ccfs.append(ccf)
    gaussian.append(gaussian_fit)
    rv_gauss.append(final_rv)
    Bobs.append(B)
    v_conv.append(vconv)
    rv_sun.append(np.mean(rv_in_file))
    error.append(np.mean(dvrms))

    # print calculation complete statement
    print('Calculation complete for', date)

# create dataframe
d = {
    'dates': dates,
    'rv_model': rv_model,
    'rv_error': error,
    'rv_sun': rv_sun,
    'ccf': ccfs,
    'gaussian': gaussian,
    'rv_gauss': rv_gauss,
    'Bobs': Bobs,
    'v_conv': v_conv
}

# save dataframe to csv using pickle
pickle_df = pd.DataFrame(data=d)
pickle_df.to_pickle(ccf_csv)



